# Vanilla GP Supervised Learning via Precision Matrix on 1D Manifold

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import torch
import gpytorch
import numpy as np
from scipy.io import loadmat
import scipy.spatial as ss
import urllib.request
import os

from time import time

from manifold_gp.models.vanilla_gp import VanillaGP

## Dataset Preprocessing

### Load & Settings

In [2]:
dataset = 'mnist' # ['protein','elevators', 'ctslice', 'mnist']
cut = 10000

if dataset == 'protein':
    data = np.loadtxt('datasets/protein.csv', delimiter=",")[:cut]
    sampled_x, sampled_y = data[:, 1:], data[:, 0]
elif dataset == 'elevators':
    data = np.array(loadmat('datasets/elevators.mat')['data'])
    sampled_x, sampled_y = data[:, :-1], data[:, -1]
elif dataset == 'ctslice':
    data = np.loadtxt('datasets/ctslice.csv', delimiter=",")[:cut]
    sampled_x, sampled_y = data[:, :-1], data[:, -1]
elif dataset == 'mnist':
    data = np.loadtxt('datasets/mnist.csv')
    sampled_x, sampled_y = data[:, 2:], data[:, 1]
    torch.manual_seed(1337)
    rand_idx = torch.randperm(sampled_x.shape[0])
    sampled_x = sampled_x[rand_idx]
    sampled_y = sampled_y[rand_idx]
    
preprocess = False
normalize_features = False
normalize_labels = True

In [3]:
if preprocess:
    # remove coincident points
    sampled_x, id_unique = np.unique(sampled_x, axis=0, return_index=True)
    sampled_y = sampled_y[id_unique]

    # cut between 0.01 and 0.99 quantile of distances
    kd_tree = ss.KDTree(sampled_x)
    v = kd_tree.query(sampled_x, k=2)[0][:, 1]
    idx = np.argsort(v)
    percentile_start = int(np.round(idx.shape[0]*0.10))
    percentile_end = int(np.round(idx.shape[0]*0.90))
    sampled_x = sampled_x[idx[percentile_start:percentile_end], :]
    sampled_y = sampled_y[idx[percentile_start:percentile_end]]
m = sampled_x.shape[0]

### Trainset & Testset

In [4]:
split = int(0.2 * m)

train_x, train_y = sampled_x[:split], sampled_y[:split]
test_x, test_y = sampled_x[split:], sampled_y[split:]

train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_y).float()
test_x = torch.from_numpy(test_x).float()
test_y = torch.from_numpy(test_y).float()

if normalize_features:
    mu_x, std_x = train_x.mean(dim=-2, keepdim=True), train_x.std(dim=-2, keepdim=True) + 1e-6
    train_x.sub_(mu_x).div_(std_x)
    test_x.sub_(mu_x).div_(std_x)
    
if normalize_labels:
    mu_y, std_y = train_y.mean(), train_y.std()
    train_y.sub_(mu_y).div_(std_y)
    test_y.sub_(mu_y).div_(std_y)

### Move Data to Device

In [5]:
train_x, train_y = train_x.contiguous(), train_y.contiguous()
test_x, test_y = test_x.contiguous(), test_y.contiguous()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
train_x, train_y = train_x.to(device), train_y.to(device)
test_x, test_y = test_x.to(device), test_y.to(device)

## Model

In [6]:
%%capture
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
    noise_prior=None  # NormalPrior(torch.tensor([0.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

kernel = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5))
# gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

model = VanillaGP(train_x, train_y, likelihood, kernel).to(device)

hypers = {
    'likelihood.noise_covar.noise': 1e-2,
    'covar_module.base_kernel.lengthscale': 0.5,
    'covar_module.outputscale': 1.0,
}
model.initialize(**hypers)

## Train

In [7]:
t0 = time()
model.vanilla_train(lr=1e-2, iter=100, verbose=True)
t1 = time()
print("Time: %.2g sec" % (t1 - t0))

Iteration: 0, Loss: -0.673, Noise Variance: 0.100, Signal Variance: 1.000, Lengthscale: 0.500
Iteration: 1, Loss: -0.677, Noise Variance: 0.100, Signal Variance: 0.997, Lengthscale: 0.504
Iteration: 2, Loss: -0.688, Noise Variance: 0.099, Signal Variance: 0.994, Lengthscale: 0.508
Iteration: 3, Loss: -0.690, Noise Variance: 0.099, Signal Variance: 0.991, Lengthscale: 0.512
Iteration: 4, Loss: -0.705, Noise Variance: 0.098, Signal Variance: 0.987, Lengthscale: 0.516
Iteration: 5, Loss: -0.706, Noise Variance: 0.098, Signal Variance: 0.984, Lengthscale: 0.520
Iteration: 6, Loss: -0.718, Noise Variance: 0.097, Signal Variance: 0.981, Lengthscale: 0.524
Iteration: 7, Loss: -0.719, Noise Variance: 0.097, Signal Variance: 0.978, Lengthscale: 0.528
Iteration: 8, Loss: -0.731, Noise Variance: 0.096, Signal Variance: 0.975, Lengthscale: 0.532
Iteration: 9, Loss: -0.727, Noise Variance: 0.096, Signal Variance: 0.972, Lengthscale: 0.536
Iteration: 10, Loss: -0.745, Noise Variance: 0.095, Signal V

Iteration: 87, Loss: -1.261, Noise Variance: 0.064, Signal Variance: 0.744, Lengthscale: 0.905
Iteration: 88, Loss: -1.268, Noise Variance: 0.064, Signal Variance: 0.742, Lengthscale: 0.910
Iteration: 89, Loss: -1.284, Noise Variance: 0.063, Signal Variance: 0.739, Lengthscale: 0.915
Iteration: 90, Loss: -1.284, Noise Variance: 0.063, Signal Variance: 0.736, Lengthscale: 0.920
Iteration: 91, Loss: -1.298, Noise Variance: 0.063, Signal Variance: 0.734, Lengthscale: 0.925
Iteration: 92, Loss: -1.301, Noise Variance: 0.062, Signal Variance: 0.731, Lengthscale: 0.930
Iteration: 93, Loss: -1.301, Noise Variance: 0.062, Signal Variance: 0.728, Lengthscale: 0.936
Iteration: 94, Loss: -1.304, Noise Variance: 0.062, Signal Variance: 0.726, Lengthscale: 0.941
Iteration: 95, Loss: -1.309, Noise Variance: 0.061, Signal Variance: 0.723, Lengthscale: 0.946
Iteration: 96, Loss: -1.325, Noise Variance: 0.061, Signal Variance: 0.721, Lengthscale: 0.951
Iteration: 97, Loss: -1.326, Noise Variance: 0.061

## Evaluation

In [8]:
%%capture
likelihood.eval()
model.eval()

## Metrics

In [9]:
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    preds_test = likelihood(model(test_x))
        
    error = test_y - preds_test.mean
    covar = preds_test.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    
    rmse = (error.square().sum()/test_y.shape[0]).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/test_y.shape[0]
    
print("RMSE: ", rmse)
print("NLL: ", nll)

RMSE:  tensor(0.0106, device='cuda:0')
NLL:  tensor(-0.8882, device='cuda:0')
